In [1]:
import pandas, os, json, pymongo, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings ; warnings.filterwarnings('ignore')

## Setup you'll need to do before you can download data from your server

In order to access the database on your server, you'll need two things. 

1. You'll need to open up a trusted line of communication from computer where you'll be analyzing the data (e.g. your desktop) to the server. 

    You can do this by modifying the following line of code to match your local configurations and entering it in from the command line: 

    `ssh -fNL 27017:localhost:27017 <USERNAME>@<DOMAIN_NAME>`

    It will ask you for your password--the same password you'd use to log onto your server


2. You'll need the the username and password from the (mongo) database itself. You'll find this information in the 'mongo_keys' file int the 'credentials/' folder on your server. 
    
    Copy a version of 'mongo_keys' onto the computer you'll be analyzing data on (we'll call that location `PATH_TO_MONGO_KEYS`)  

## Step by step code to connect with your server 

In [2]:
PATH_TO_MONGO_KEYS = '<YOUR_PATH>'

In [3]:
#  to 'mongo_keys' which you should have in your 'credentials/' folder on the server
mongo_keys = os.path.join(PATH_TO_MONGO_KEYS, 'mongo_keys') 

# load our mongo access keys
access_info = json.load(open(os.path.join(credential_location, mongo_keys)))

# set up the 'tunnel' we'll be using to access the server (which we opened above)
mongo_tunnel = 'mongodb://' + access_info['user'] + ':' + access_info['pwd'] + '@127.0.0.1'

# conect to the mongo client on the server
connection_to_server = pymongo.MongoClient(mongo_tunnel)

Here, "connection_to_server" is the umbrella connection that you'll use to set up a connection to the specific `database` and `collection` where your data have been stored. We know what database and collection were used to store the "hello_world" data from the `server_side_psych/experiment_setup/hello_world/app.js` file. 

Line 83 specifies the database (`db`) and collection (`collection`): 
 
 
`var collection = client.db('hello_world_database').collection('comments')`

Let's use those database and collection names: 

In [4]:
# define database we want to connect to
mongo_database = 'hello_world_database'

# define collection we want to connect to
mongo_collection = 'comments'

In [5]:
# connect to the database
data_base = connection_to_server[mongo_database]

# connect to the collection
collection = data_base[mongo_collection]

Let's collect all the documents you've saved from the 'hello_world' demo you just worked through: 

In [6]:
collection_documents = [i for i in collection.find()]

And let's just look at the first thing you saved: 

In [7]:
collection_documents[0] # requires that there is at least one document saved to the database :) 

We Can also look at all the databases on the server: 

In [8]:
connection_to_server.list_database_names()

Let's look at each of the collections in these databases, and count the number of documents within them: 

In [9]:
for d in [i for i in connection_to_server.list_database_names() if i not in ['admin', 'config']]:
    print(d, ':')
    for c in connection[d].list_collection_names(): 
        print( "\t The '%s' collection  has %d documents"%(c,connection[d][c].count() ))